In [1]:
import modin.pandas as pd
import numpy as np

In [2]:
# Quickly check annotation progress.
pd.read_parquet("wforms-ann.parquet")["status"].value_counts(dropna=False)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-31 15:59:40,707	INFO worker.py:1636 -- Started a local Ray instance.


status
-1.0    11102
 1.0      346
 NaN       76
Name: count, dtype: int64

In [3]:
annos = (
    pd.read_parquet("wforms-bat.parquet")
    .join(pd.read_parquet("wforms-occ.parquet", columns=["tot"]), how="left")
    .join(pd.read_parquet("wforms-ann.parquet"), how="left")
)

# Previously attested forms and hashtags get a special status value.
annos.loc[annos["tt_att"], "status"] = +np.inf
annos.loc[annos.index.str.startswith("#"), "status"] = -np.inf

annos[annos["fst_batch"] | annos["snd_batch"]]["status"].value_counts(dropna=False)

status
 inf    14138
-1.0     7709
-inf     4619
 1.0      346
 NaN       78
Name: count, dtype: int64

In [4]:
# Normalize categories.
annos.loc[annos["category"].isna(), "category"] = None
annos["category"] = (
    annos["category"]
    .str.replace("?", "")
    .str.replace("-1", "")
    .str.replace(";", ",")
    .str.replace(" ", "")
    .str.replace("variantgrafica", "variantegrafica")
    .str.replace("enfacigrafica", "enfasigrafica")
)
annos.loc[annos["category"] == "", "category"] = None

In [5]:
# Print category usage statistics (mind the double counting from multi-category forms!)
with pd.option_context('display.max_rows', None):
    focus = annos["fst_batch"] | annos["snd_batch"]
    display(annos[focus]["category"].value_counts())
    display(annos[focus]["category"].str.split(",").explode().value_counts())
    display(annos[focus]["category"].str.split(",").dropna().map(lambda l: l[0]).value_counts())

category
variantegrafica          109
univerbazione             48
suffissazione             45
forestierismo             40
macedonia                 33
prestitoadattato          24
alterazione               17
prefissazione              8
acronimo                   6
transcategorizzazione      6
composizione               3
deonimico                  3
ridefinizione              2
deacronimico               1
tmesi                      1
Name: count, dtype: int64

category
variantegrafica          109
univerbazione             48
suffissazione             45
forestierismo             40
macedonia                 33
prestitoadattato          24
alterazione               17
prefissazione              8
acronimo                   6
transcategorizzazione      6
composizione               3
deonimico                  3
ridefinizione              2
deacronimico               1
tmesi                      1
Name: count, dtype: int64

category
variantegrafica          109
univerbazione             48
suffissazione             45
forestierismo             40
macedonia                 33
prestitoadattato          24
alterazione               17
prefissazione              8
acronimo                   6
transcategorizzazione      6
composizione               3
deonimico                  3
ridefinizione              2
deacronimico               1
tmesi                      1
Name: count, dtype: int64

In [6]:
with pd.option_context('display.max_rows', 1000):
    focus = annos["status"].isin([0,1])
    display(annos[focus].sort_values("tot", ascending=False))

,tt_att,fst_batch,snd_batch,tot,attestation,category,notes,status
wf,,,,,,,,
rdc,False,True,False,4987,None,acronimo,None,1.0
csx,False,True,False,1064,None,variantegrafica,centro-sinistra,1.0
buonagiornata,False,True,False,526,None,univerbazione,None,1.0
twitteri,False,True,False,506,None,suffissazione,None,1.0
piddini,False,True,False,496,None,suffissazione,None,1.0
tuitteri,False,True,False,393,None,suffissazione,None,1.0
pidioti,False,True,False,377,None,macedonia,None,1.0
reel,False,False,True,315,None,forestierismo,None,1.0
lho,False,True,False,307,None,univerbazione,l'ho,1.0
